#### 1. 초기 설정 및 라이브러리 임포트
이 셀에서는 필요한 라이브러리를 임포트하고, 스크래핑에 필요한 초기 변수들을 설정합니다.

In [ ]:
# %%
import requests
from bs4 import BeautifulSoup
import os
import re
from datetime import datetime
import time

# 스크래핑할 기본 URL 설정
BASE_URL_TEMPLATE = "https://business.nikkei.com/atcl/gen/19/00461/?TOC={page_num}"
# 초기 페이지 번호
initial_page_num = 1
# 초기 키워드 설정
initial_keyword = "business"
# 초기 ID 번호 (문자열 형식)
initial_id_num = "001"

# 저장할 디렉토리 설정: 코드가 위치한 루트 디렉토리의 'data_article' 폴더
output_dir = os.path.join(os.getcwd(), "data_article")

# 출력 디렉토리 생성 (없을 경우)
os.makedirs(output_dir, exist_ok=True)

# 봇으로 인식되지 않기 위한 User-Agent 헤더 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

print(f"스크래핑 시작을 위한 초기 설정이 완료되었습니다.")
print(f"기본 URL 템플릿: {BASE_URL_TEMPLATE}")
print(f"초기 페이지 번호: {initial_page_num}")
print(f"초기 키워드: {initial_keyword}")
print(f"저장될 디렉토리: {output_dir}")
print(f"사용될 User-Agent: {headers['User-Agent']}")

# %%

#### 2. 페이지 URL에서 기사 링크 추출 함수
이 셀에서는 주어진 페이지 URL에서 모든 기사 링크를 추출하는 함수를 정의합니다.

In [ ]:
# %%
def get_article_links(page_url):
    """
    주어진 URL에서 기사 링크를 추출합니다.
    """
    print(f"\n페이지에서 기사 링크를 추출합니다: {page_url}")
    try:
        # headers를 요청에 포함
        response = requests.get(page_url, headers=headers)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
    except requests.exceptions.RequestException as e:
        print(f"URL 접속 중 오류 발생: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # <div class="p-articleList_list"> 컨테이너 찾기
    article_list_container = soup.find('div', class_='p-articleList_list')
    
    if not article_list_container:
        print("기사 목록 컨테이너를 찾을 수 없습니다.")
        return []
    
    # 모든 <a class="p-articleList_item_link"> 태그 조회
    links = article_list_container.find_all('a', class_='p-articleList_item_link')
    
    article_urls = []
    for link in links:
        href = link.get('href')
        if href:
            # 상대 경로를 절대 경로로 변환 (필요한 경우)
            if not href.startswith('http'):
                # 기본적인 도메인 부분을 가져와서 합쳐줍니다.
                # 이 예제에서는 business.nikkei.com을 사용합니다.
                href = f"https://business.nikkei.com{href}" 
            article_urls.append(href)
            
    print(f"총 {len(article_urls)}개의 기사 링크를 찾았습니다.")
    return article_urls

# %%

#### 3. 단일 기사 페이지 스크래핑 및 저장 함수
이 셀에서는 각 기사 페이지로 이동하여 제목, 날짜, 본문 내용을 스크래핑하고 마크다운 파일로 저장하는 함수를 정의합니다.

In [ ]:
# %%
def scrape_article_and_save(article_url, current_keyword, current_id_num):
    """
    단일 기사 페이지를 스크래핑하고 마크다운 파일로 저장합니다.
    """
    print(f"\n기사 스크래핑 중: {article_url}")
    try:
        # headers를 요청에 포함
        response = requests.get(article_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"기사 페이지 접속 중 오류 발생: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # 기사 컨테이너 찾기
    article_container = soup.find('div', class_='p-article js-stickyArticleMenu_container')
    if not article_container:
        print("기사 컨테이너를 찾을 수 없습니다.")
        return None

    # 제목 스크래핑
    title_tag = article_container.find('p', class_='p-article_header_title')
    title = title_tag.get_text(strip=True) if title_tag else "제목 없음"
    print(f"  제목: {title}")

    # 작성 날짜 스크래핑
    datetime_str = "unknown_date"
    date_item = soup.find('li', class_='p-article_header_meta_item -date')
    if date_item:
        time_tag = date_item.find('time')
        if time_tag and 'datetime' in time_tag.attrs:
            datetime_str = time_tag['datetime']
            print(f"  작성 날짜: {datetime_str}")
        else:
            print("  작성 날짜 (time 태그 또는 datetime 속성)를 찾을 수 없습니다.")
    else:
        print("  작성 날짜 (li 태그)를 찾을 수 없습니다.")

    # 본문 내용 스크래핑
    # 모든 <p> 태그의 문자열 내용들을 가져오고, <b> 태그 등은 무시
    paragraphs = article_container.find_all('p')
    article_text = []
    for p in paragraphs:
        # 태그 내의 모든 텍스트를 가져오되, 자식 태그의 텍스트도 포함 (strip=True로 공백 제거)
        # BeautifulSoup의 .get_text()는 기본적으로 자식 태그를 무시하지 않습니다.
        # 단순히 텍스트만 가져오므로 <b> 태그는 자동으로 무시됩니다.
        cleaned_text = p.get_text(strip=True)
        if cleaned_text: # 빈 줄은 건너뛰기
            article_text.append(cleaned_text)
            
    full_article_content = "\n".join(article_text)
    
    # 파일 이름 생성
    # datetime_str에서 연-월-일만 추출 (YYYY-MM-DD 형식)
    file_date = datetime_str
    if re.match(r"\d{4}-\d{2}-\d{2}", datetime_str): # "YYYY-MM-DD" 형식인지 확인
        file_date = datetime_str
    elif "T" in datetime_str: # "YYYY-MM-DDTHH:MM:SS" 형식일 경우
        file_date = datetime_str.split('T')[0]
    
    filename = f"article_nikkei_{current_keyword}_{file_date}_{current_id_num}.md"
    filepath = os.path.join(output_dir, filename)

    # 마크다운 파일 내용 작성
    markdown_content = f"### {title}\n\n"
    markdown_content += full_article_content

    # 파일 저장
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        print(f"  기사 저장 완료: {filepath}")
        return True
    except IOError as e:
        print(f"파일 저장 중 오류 발생: {e}")
        return False

# %%

#### 4. 메인 스크래핑 루프
이 셀에서는 전체 스크래핑 과정을 제어하는 메인 루프를 실행합니다.

In [ ]:
current_page_num = initial_page_num
current_keyword = initial_keyword
current_id_int = int(initial_id_num) # ID 번호를 정수로 관리하여 증가시키기 위함

max_pages_to_scrape = 5 # 스크래핑할 최대 페이지 수 (테스트를 위해 제한)
# 무한 루프를 원한다면 이 변수를 주석 처리하거나 매우 큰 값으로 설정하세요.

while True:
    page_url = BASE_URL_TEMPLATE.format(page_num=current_page_num)
    print(f"\n--- 현재 페이지: {page_url} (페이지 번호: {current_page_num}) ---")

    article_links = get_article_links(page_url)

    if not article_links:
        print(f"더 이상 기사 링크를 찾을 수 없거나 페이지가 존재하지 않습니다. 스크래핑을 종료합니다.")
        break

    for link in article_links:
        # ID 번호를 3자리 문자열로 포맷팅
        formatted_id_num = f"{current_id_int:03d}"
        
        success = scrape_article_and_save(link, current_keyword, formatted_id_num)
        if success:
            current_id_int += 1 # 성공적으로 저장되면 ID 증가
        else:
            print(f"경고: {link} 스크래핑에 실패했습니다. 다음 기사로 넘어갑니다.")
        
        # 과도한 요청을 방지하기 위한 지연 시간
        time.sleep(1) 

    print(f"\n페이지 {current_page_num}의 모든 기사 스크래핑을 완료했습니다.")
    
    current_page_num += 1
    
    # 최대 페이지 스크래핑 제한 확인
    if 'max_pages_to_scrape' in locals() and current_page_num > initial_page_num + max_pages_to_scrape -1 :
        print(f"설정된 최대 스크래핑 페이지 수({max_pages_to_scrape} 페이지)에 도달했습니다. 스크래핑을 종료합니다.")
        break

    # 다음 페이지로 넘어가기 전 충분한 지연 시간
    time.sleep(2)

print("\n--- 모든 스크래핑 작업이 완료되었습니다. ---")
